# Cache CDEC stations information

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cdec_maps import cdec
from cdec_maps import cdec
import diskcache as dc

import glob
import datetime
import pandas as pd

In [ ]:
cdec.read_stations_with_meta()

In [ ]:
#FSW failed stations.ID[262], tables are too many
#POH failed stations.ID[475]
#HOP failed
#GDV
#HIB
#MVV
#NRN
#OCS

In [ ]:
selected_stations = stations.query('ID in ("FPT","YBY")')
selected_stations

In [ ]:
station_with_meta.dtypes

In [ ]:
selected_stations.dtypes

In [ ]:
station_with_meta.ID in selected_stations.ID

In [ ]:
station_with_meta.join(selected_stations[['ID']].set_index('ID'),on='ID',how='inner')

In [ ]:
station_with_meta.join(selected_stations,on="ID")

In [ ]:
station_with_meta[station_with_meta['Sensor Description'].str.match('PRECIP')]['Sensor Number'].unique()

In [ ]:
stations_with_precip_accumulated = station_with_meta.query('`Sensor Number`==2')

In [ ]:
import hvplot.pandas

In [ ]:
stations_with_precip_accumulated

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
stations_with_precip_accumulated.hvplot.points('Longitude','Latitude',geo=True,tiles='CartoLight',hover_cols='all')

In [ ]:
scurrent = stations_with_precip_accumulated[stations_with_precip_accumulated['Data Available'].str.contains('present')]

In [ ]:
scurrent.groupby(['ID','Duration']).count()

In [ ]:
scurrent.query('`Duration`=="(daily)"')

In [ ]:
def load_data_from_cache(cache, sid, sensor_number, duration_code):
    return cache[f'{sid}_{sensor_number}_{duration_code}']

In [ ]:
def read_station_data_and_cache(cache, sid, sensor_number, duration_code):
    r = cdec.Reader()
    key = f'{sid}_{sensor_number}_{duration_code}'
    if key in cache:
        data = cache[key]
        sdate = data.last_valid_index().strftime('%Y-%m-%d')
        edate = pd.Timestamp.now().strftime('%Y-%m-%d')
        if sdate == edate:
            return cache[key] # return cache, (no update cache)
        else: # update extra days
            data_ext = r.read_station_data(sid, sensor_number, duration_code, sdate, edate)
            data = data.combine_first(data_ext)
    else:
        data = r.read_entire_station_data_for(sid, sensor_number, duration_code)
    cache[key] = data # update cache
    return data

In [ ]:
data = read_station_data_and_cache(cache, 'SWX', 2, 'D')

In [ ]:
precip_daily = scurrent.query('`Duration`=="(daily)"')

In [ ]:
for i, r in tqdm.tqdm(precip_daily.iloc[149:].iterrows(), total=len(precip_daily.iloc[149:])):
    read_station_data_and_cache(cache, r['ID'], r['Sensor Number'], cdec.DURATION_MAP[r['Duration']])